# 第6章 缺失数据

#### 在接下来的两章中，会接触到数据预处理中比较麻烦的类型，即缺失数据和文本数据（尤其是混杂型文本）
#### Pandas在步入1.0后，对数据类型也做出了新的尝试，尤其是Nullable类型和String类型，了解这些可能在未来成为主流的新特性是必要的

In [20]:
import pandas as pd
import numpy as np

## 五、问题与练习

### 1. 问题

#### 【问题一】 如何删除缺失值占比超过25%的列？  
按列计算缺失值比例 `df.isna().sum()/df.shape[0]`，选出`df.loc[:,(df.isna().sum()/df.shape[0])<0.25]`
#### 【问题二】 什么是Nullable类型？请谈谈为什么要引入这个设计？
- Nullable类型是一组将缺失值统一为pd.NA的衍生数据类型  
- **引入原因**：
    - Pandas早期版本中涉及到缺失值有np.nan、None、NaT等三种，运算性质、使用方法各有细微不同，引入pd.NA试图进行统一
    - 引入string，以区分开原本含糊不清的object类型  

#### 【问题三】 对于一份有缺失值的数据，可以采取哪些策略或方法深化对它的了解？  
- 调用.info()查看缺失值数量
- 在缺失项占比不大的情况（<25%）下，尝试使用简单填充/差值方式进行填补
- 否则，对某些含缺失值较多时行或列进行剔除

### 2. 练习

#### 【练习一】现有一份虚拟数据集，列类型分别为string/浮点/整型，请解决如下问题：
#### （a）请以列类型读入数据，并选出C为缺失值的行。

In [21]:
#(a)
df = pd.read_csv('data/Missing_data_one.csv').convert_dtypes()
df[df['C'].isna()]

,A,B,C
1,not_NaN,0.700,<NA>
5,not_NaN,0.972,<NA>
11,not_NaN,0.736,<NA>
19,not_NaN,0.684,<NA>
21,not_NaN,0.913,<NA>


#### （b）现需要将A中的部分单元转为缺失值，单元格中的最小转换概率为25%，且概率大小与所在行B列单元的值成正比。

In [22]:
maxB = df.B.max()
minB = df.B.min()
print(maxB/minB*.25)

df['A'] = df[["A","B"]].apply(lambda x: np.nan if np.random.rand()<.25*x["B"]/minB else x["A"],axis=1).convert_dtypes()
df.head()

0.4850894632206759


,A,B,C
0,not_NaN,0.922,4
1,<NA>,0.700,<NA>
2,<NA>,0.503,8
3,not_NaN,0.938,4
4,not_NaN,0.952,10


#### 【练习二】 现有一份缺失的数据集，记录了36个人来自的地区、身高、体重、年龄和工资，请解决如下问题：
#### （a）统计各列缺失的比例并选出在后三列中至少有两个非缺失值的行。

In [23]:
df2 = pd.read_csv('data/Missing_data_two.csv').convert_dtypes()
df2.head()
display(df2.isna().sum()/df2.shape[0])
display(df2[df2[['体重','年龄','工资']].isna().sum(1)>1])

编号    0.000000
地区    0.000000
身高    0.000000
体重    0.222222
年龄    0.250000
工资    0.222222
dtype: float64

,编号,地区,身高,体重,年龄,工资
2,3,C,169.09,62.18,<NA>,<NA>
11,12,A,202.56,92.30,<NA>,<NA>
12,13,C,177.37,NaN,79,<NA>
14,15,C,199.11,89.20,<NA>,<NA>
26,27,B,158.28,NaN,51,<NA>
32,33,C,181.01,NaN,<NA>,13021
33,34,A,196.67,87.00,<NA>,<NA>


In [24]:
df2.loc[:,(df2.isna().sum()/df2.shape[0])<0.25]

,编号,地区,身高,体重,工资
0,1,A,157.50,NaN,15905
1,2,B,202.00,91.80,<NA>
2,3,C,169.09,62.18,<NA>
3,4,A,166.61,59.95,5434
4,5,B,185.19,NaN,4242
5,6,A,187.13,78.42,13959
6,7,C,163.81,57.43,6533
7,8,A,183.80,75.42,19779
8,9,B,179.67,71.70,8608
9,10,C,186.08,77.47,12433


#### （b）请结合身高列和地区列中的数据，对体重进行合理插值。

In [25]:
df_w = df2.copy()
grouped = df_w.groupby('地区')
#参照答案实现
for name,group in grouped:
    df_w.loc[group.index,'体重'] = group[['身高','体重']].sort_values(by='身高').interpolate()['体重'].round(decimals=2)
df_w.head(10)


,编号,地区,身高,体重,年龄,工资
0,1,A,157.50,53.58,47,15905
1,2,B,202.00,91.80,25,<NA>
2,3,C,169.09,62.18,<NA>,<NA>
3,4,A,166.61,59.95,77,5434
4,5,B,185.19,81.75,62,4242
5,6,A,187.13,78.42,55,13959
6,7,C,163.81,57.43,43,6533
7,8,A,183.80,75.42,48,19779
8,9,B,179.67,71.70,65,8608
9,10,C,186.08,77.47,65,12433
